## - Combine all data 

In [21]:
import pandas as pd
from os import listdir

path = '../data/'
files = listdir('../data/')
df = pd.DataFrame(columns=["url", "query", "text"])

for f in files:
    temp = pd.read_csv(path + f)
    if 'article-name' in temp.columns:
        temp.rename(columns={'article-name':'name','article-url':'url','content':'text','keyword':'query'}, inplace=True)
    if len(temp) < 1:
        continue
    df = df.append(temp)
df.drop(['Unnamed: 0', 'name'], inplace=True, axis=1)

In [22]:
df

,query,text,url
0,APT30,Today FireEye released a report on a threat gr...,https://www.fireeye.com//blog/executive-perspe...
1,APT30,APT30: The Mechanics\n Behind a Decade Long C...,https://www.fireeye.com//current-threats/apt-g...
0,APT29,\nThe sophisticated threat actor COZY BEAR wa...,https://www.crowdstrike.com/blog/who-is-cozy-b...
0,APT38,"Today, we are releasing details on the threat ...",https://www.fireeye.com//blog/threat-research/...
0,APT12,+2\n2\n0\n\n2 Votes\n\n\n\n\nSymantec Official...,https://www.symantec.com/connect/blogs/yet-ano...
0,APT15,APT Attackers Target Japanese Word Processor Z...,https://www.securityweek.com/apt-attackers-tar...
1,APT15,China-Linked Spies Used New Malware in U.K. Go...,https://www.securityweek.com/china-linked-spie...
2,APT15,China-Linked Spies Used New Malware in U.K. Go...,https://www.securityweek.com/china-linked-spie...
0,APT34,Less than a week after Microsoft issued a patc...,https://www.fireeye.com//blog/threat-research/...
0,APT35,Iranian Hackers Impersonate Israeli Security F...,https://www.securityweek.com/iranian-hackers-i...


## - data preprocessing
    1. stop word removal
    2. lower case letters
    3. non ascii character removal

In [23]:
from nltk.corpus import stopwords
import re
stop = stopwords.words('english')

def normalize_text(text):
    norm_text = text.lower()
    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')
    # Pad punctuation with spaces on both sides
    norm_text = re.sub(r"([\.\",\(\)!\?;:])", " \\1 ", norm_text)
    return norm_text

def remove_stop_words(text):
    return " ".join([item.lower() for item in text.split() if item not in stop])

def remove_non_ascii(text):
    return ''.join(["" if ord(i) < 32 or ord(i) > 126 else i for i in text])

df['text'] = df['text'].apply(remove_non_ascii)
df['text'] = df['text'].apply(normalize_text)
df['text'] = df['text'].apply(remove_stop_words)
df["text"] = df['text'].str.replace('[^\w\s]','')

## - a simple word2vec model
    In this section we apply simple word to vec model to tokenized data.

In [24]:
from gensim.models import Word2Vec
from nltk import word_tokenize

In [25]:
df['tokenized_text'] = df.apply(lambda row: word_tokenize(row['text']), axis=1)

In [26]:
model = Word2Vec(df['tokenized_text'], size=100)

In [27]:
for num in [1, 3, 5, 10, 12, 16, 17, 18, 19, 28, 29, 30, 32, 33, 34, 37, 38]:
    term = "apt%s"%str(num)
    if term in model.wv.vocab:
        print("Most similar words for %s"%term)
        for t in model.most_similar(term): print(t)
        print('\n')

Most similar words for apt1
('china', 0.9991138577461243)
('mandiant', 0.9987292289733887)
('lab', 0.9985379576683044)
('kaspersky', 0.9985246658325195)
('labs', 0.9983357191085815)
('military', 0.9983073472976685)
('defense', 0.9981494545936584)
('2017', 0.9981141686439514)
('according', 0.9980982542037964)
('iranian', 0.9977273941040039)


Most similar words for apt3
('tracked', 0.9979130029678345)
('cozy', 0.9970728158950806)
('team', 0.9964618682861328)
('dragonfly', 0.9960741400718689)
('strontium', 0.9953812956809998)
('operation', 0.9952284097671509)
('behind', 0.9945781826972961)
('hackers', 0.9930994510650635)
('campaign', 0.99244624376297)
('cyberespionage', 0.9922792911529541)


Most similar words for apt10
('sophisticated', 0.9997135400772095)
('entities', 0.9992278218269348)
('operating', 0.9990057945251465)
('kitten', 0.9989932179450989)
('region', 0.9989686608314514)
('naikon', 0.9988833665847778)
('launched', 0.9988541603088379)
('analyzed', 0.9988355040550232)
('apt32'

/home/born-2-code/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


### here we got one interesting result for apt17 as apt28
    but for all other word2vec results we observe that we are getting names like malware, attackers, groups, backdoor in the most similar items.  
    It might be the case that the names of attacker groups are ommited because they are phrases instead simple words.

## - word2vec with bigram phrases
    here we try to find bigram phrases from the dataset and apply word2vec model to it

In [28]:
from gensim.models import Phrases
from collections import Counter

In [29]:
bigram = Phrases()

In [30]:
bigram.add_vocab(df['tokenized_text'])

In [31]:
bigram_counter = Counter()
for key in bigram.vocab.keys():
    if len(key.split("_")) > 1:
        bigram_counter[key] += bigram.vocab[key]

for key, counts in bigram_counter.most_common(20):
    print '{0: <20} {1}'.format(key.encode("utf-8"), counts)

cyber_security       353
security_conference  334
ics_cyber            334
document_getelementsbytagname 163
comjsplusone_js      163
conference_singapore 163
google_comjsplusone  163
script_0             163
ciso_forum           163
forum_half           163
document_createelement 163
po_src               163
apis_google          163
textjavascript_po    163
type_textjavascript  163
po_async             163
var_po               163
parentnode_insertbefore 163
async_true           163
po_type              163


In [32]:
bigram_model = Word2Vec(bigram[df['tokenized_text']], size=100)

In [33]:
for num in [1, 3, 5, 10, 12, 16, 17, 18, 19, 28, 29, 30, 32, 33, 34, 37, 38]:
    term = "apt%s"%str(num)
    if term in bigram_model.wv.vocab:
        print("Most similar words for %s"%term)
        for t in bigram_model.most_similar(term, topn=20): print(t)
        print('\n')

Most similar words for apt1
(u'systems', 0.9999414086341858)
(u'likely', 0.9999133944511414)
(u'could', 0.9999118447303772)
(u'actors', 0.9999102354049683)
(u'two', 0.9999037384986877)
(u'would', 0.9999027252197266)
(u'data', 0.9999006986618042)
(u'threat_actors', 0.9998990297317505)
(u'vulnerabilities', 0.9998963475227356)
(u'based', 0.9998944401741028)
(u'2017', 0.9998939037322998)
(u'different', 0.9998932480812073)
(u'compromised', 0.9998882412910461)
(u'recently', 0.9998868703842163)
(u'often', 0.9998851418495178)
(u'additional', 0.999885082244873)
(u'says', 0.9998839497566223)
(u'seen', 0.9998784065246582)
(u'past', 0.9998762607574463)
(u'symantec', 0.9998751282691956)


Most similar words for apt3
(u'actor', 0.9998279809951782)
(u'multiple', 0.9998132586479187)
(u'believe', 0.9997843503952026)
(u'experts', 0.9997732639312744)
(u'also_known', 0.9997693300247192)
(u'2017', 0.9997662305831909)
(u'north_korean', 0.999755322933197)
(u'last_year', 0.9997542500495911)
(u'likely', 0.9997

/home/born-2-code/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


### After applying bigram phrases still we cannot see the desired results. 

## Doc2Vec model

In [34]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [35]:
df_doc = df[['query', 'text']]

In [36]:
df_doc = df_doc.groupby(['query'],as_index=False).first()

In [37]:
tagged_data = [TaggedDocument(words=word_tokenize(df_doc['text'].iloc[i]), tags=[df_doc['query'].iloc[i]]) for i in range(len(df_doc))]

In [38]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/home/born-2-code/.local/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  app.launch_new_instance()


Model Saved


In [39]:
for num in [1, 3, 5, 10, 12, 16, 17, 18, 19, 28, 29, 30, 32, 33, 34, 37, 38]:
    term = "APT%s"%str(num)
    if term in list(df_doc['query']):
        print(term)
        print(model.docvecs.most_similar(term))
        print('\n')

APT1
[('APT28', 0.7839778065681458), ('APT29', 0.7835925817489624), ('APT15', 0.7258604168891907), ('APT27', 0.7226715087890625), ('APT18', 0.7201363444328308), ('APT38', 0.7148082256317139), ('APT12', 0.70294189453125), ('APT17', 0.6991536617279053), ('APT3', 0.6950184106826782), ('APT35', 0.6926254034042358)]


APT3
[('APT17', 0.7516874670982361), ('APT37', 0.7471446394920349), ('APT29', 0.7417770028114319), ('APT38', 0.7213995456695557), ('APT28', 0.7209811210632324), ('APT12', 0.6950358748435974), ('APT1', 0.6950184106826782), ('APT16', 0.6718595027923584), ('APT35', 0.6598621606826782), ('APT34', 0.6495394706726074)]


APT10
[('APT35', 0.8204693794250488), ('APT29', 0.7824466228485107), ('APT17', 0.731864333152771), ('APT33', 0.7253490090370178), ('APT27', 0.7108751535415649), ('APT18', 0.7073136568069458), ('APT15', 0.686213493347168), ('APT16', 0.6819599866867065), ('APT28', 0.6761863231658936), ('APT34', 0.6420496702194214)]


APT12
[('APT15', 0.8837562799453735), ('APT30', 0.7

In [46]:
model.similar_by_word('apt12')

/home/born-2-code/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('belonging', 0.9390033483505249),
 ('possibly', 0.8240631818771362),
 ('wekby', 0.7597745656967163),
 ('hotel', 0.7557625770568848),
 ('ga', 0.7460821270942688),
 ('turla', 0.7373042702674866),
 ('sword', 0.7202558517456055),
 ('platform', 0.7161414623260498),
 ('significant', 0.7154147028923035),
 ('succeeded', 0.7152796983718872)]

## doc2vec with each sentence as a document